In [1]:
import pickle 
import numpy as np 
import pandas as pd
import networkx as nx
from tqdm import tqdm
import time
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
import community
import collections

from src.utils.data_utils import *
from src.utils.general_utils import *
from src.utils.mapping import *
from src.utils.langue import *

In [2]:
movie_path = 'data/Movie.pkl'
actors_path = 'data/Actor.pkl'

with open(movie_path, 'rb') as file:
    Movie = pickle.load(file)
Movie = Movie.reset_index()
with open(actors_path, 'rb') as file:
    Actor = pickle.load(file)

C:\Users\Thibault Schiesser\AppData\Local\Temp\ipykernel_2924\2668311966.py:5: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  Movie = pickle.load(file)


In [3]:
nombre_entrees = Movie.Movie_languages.str.contains(',', na=False).sum()

In [4]:
nombre_entrees

np.int64(7888)

In [5]:
Movie.Movie_languages.shape

(61042,)

In [6]:
a = create_actor_language_dataset(Movie, Actor, 5)

We lose 13.58% of the dataset of movies with this operation.


In [7]:
presence = create_cross_language(a)
compte = create_cross_language_count(a)

In [8]:
presence['English Language']

,/m/0bxl4_7,/m/0bxl4__,/m/0bxl4_h,/m/0bxl4_r,/m/0bxl4z5,/m/0bxl4z_,/m/0bxl4zs,/m/0bxl506,/m/0k6ks2,/m/0k6ks8,...,/m/05qrj_,/m/01f46l,/m/0k19q1b,/m/02sty,/m/04jhqtq,/m/05b349c,/m/0bzh52,/m/0n28yxr,/m/01m54f8,sum
Movie_languages,,,,,,,,,,,,,,,,,,,,,
English Language,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,61992.0
Hindi Language,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,177.0
Spanish Language,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,426.0
Italian Language,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,734.0
French Language,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,749.0


In [12]:
compte['English Language'][compte['English Language'].nb_movie>5]

Movie_languages,English Language,Hindi Language,Spanish Language,Italian Language,French Language,nb_movie
/m/0d_65z,6.0,0.0,0.0,0.0,0.0,6.0
/m/03c0srq,6.0,0.0,0.0,0.0,0.0,6.0
/m/03cbh23,6.0,0.0,0.0,0.0,0.0,6.0
/m/0krxsd,7.0,0.0,0.0,0.0,0.0,7.0
/m/02z7tbt,8.0,0.0,0.0,0.0,0.0,8.0
...,...,...,...,...,...,...
/m/026t8jg,1.0,0.0,5.0,1.0,0.0,7.0
/m/026ksyr,1.0,8.0,0.0,0.0,0.0,9.0
/m/04y0hj,1.0,27.0,0.0,0.0,0.0,28.0
/m/0b781h6,1.0,10.0,0.0,0.0,0.0,11.0
